<a href="https://colab.research.google.com/github/A7mad-Magdy77/01/blob/main/Advanced_Browser_Agent_Marktechpost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get update -qq
!apt-get install -y -qq chromium-browser chromium-chromedriver fonts-liberation
!pip install -qq playwright python-dotenv langchain-google-generative-ai browser-use
!playwright install

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
ERROR: Could not find a version that satisfies the requirement langchain-google-generative-ai (from versions: none)
ERROR: No matching distribution found for langchain-google-generative-ai
Playwright Host validation warning: 
╔══════════════════════════════════════════════════════╗
║ Host system is missing dependencies to run browsers. ║
║ Missing libraries:                                   ║
║     libwoff2dec.so.1.0.2                             ║
║     libgstgl-1.0.so.0                                ║
║     libgstcodecparsers-1.0.so.0                      ║
║     libavif.so.13                                    ║
║     libharfbuzz-icu.so.0                             ║
║     libenchant-2.so.2                                ║
║     libsecret-1.so.0                                 ║
║     libhyphen

In [ ]:
import os
import asyncio
from getpass import getpass
from pydantic import SecretStr
from langchain_google_genai import ChatGoogleGenerativeAI
from browser_use import Agent, Browser, BrowserContextConfig, BrowserConfig
from browser_use.browser.browser import BrowserContext

In [ ]:
os.environ["ANONYMIZED_TELEMETRY"] = "false"

In [ ]:
async def setup_browser(headless: bool = True):
    browser = Browser(config=BrowserConfig(headless=headless))
    context = BrowserContext(
        browser=browser,
        config=BrowserContextConfig(
            wait_for_network_idle_page_load_time=5.0,
            highlight_elements=True,
            save_recording_path="./recordings",
        )
    )
    return browser, context

In [ ]:
async def agent_loop(llm, browser_context, query, initial_url=None):
    initial_actions = [{"open_tab": {"url": initial_url}}] if initial_url else None
    agent = Agent(
        task=query,
        llm=llm,
        browser_context=browser_context,
        use_vision=True,
        generate_gif=False,
        initial_actions=initial_actions,
    )
    result = await agent.run()
    return result.final_result() if result else None

In [ ]:
async def main():
    raw_key = getpass("Enter your GEMINI_API_KEY: ")

    os.environ["GEMINI_API_KEY"] = raw_key

    api_key = SecretStr(raw_key)
    model_name = "gemini-2.5-flash-preview-04-17"

    llm = ChatGoogleGenerativeAI(model=model_name, api_key=api_key)

    browser, context = await setup_browser(headless=True)

    try:
        while True:
            query = input("\nEnter prompt (or leave blank to exit): ").strip()
            if not query:
                break
            url = input("Optional URL to open first (or blank to skip): ").strip() or None

            print("\n🤖 Running agent…")
            answer = await agent_loop(llm, context, query, initial_url=url)
            print("\n📊 Search Results\n" + "-"*40)
            print(answer or "No results found")
            print("-"*40)
    finally:
        print("Closing browser…")
        await browser.close()

await main()

Enter your GEMINI_API_KEY: ··········

Enter prompt (or leave blank to exit): What’s the current price of Bitcoin? Please grab it from coinmarketcap.com and show me the number
Optional URL to open first (or blank to skip): 

🤖 Running agent…



📊 Search Results
----------------------------------------
The current price of Bitcoin on coinmarketcap.com is $84,446.56
----------------------------------------
